In [1]:
##### AGGREGATION #####

In [2]:
print('Starting aggregation...')

Starting aggregation...


In [3]:
### import libraries
import pandas as pd
import numpy as np
from datetime import datetime,date

start_time = datetime.now()
print('Start time: ', start_time)

Start time:  2019-03-08 18:52:14.849274


In [4]:
#### SELECT INPUT AND OUTPUT FILES

In [5]:
input_file = '3_day_sample_cleaned_and_mapped.tsv.gz'
output_file = '3_day_sample_aggregated.tsv.gz'
#input_file = '6_week_sample_cleaned_and_mapped.tsv.gz'
#output_file = '6_week_sample_aggregated.tsv.gz'
#input_file = '12_week_sample_cleaned_and_mapped.tsv.gz'
#output_file = '12_week_sample_aggregated.tsv.gz'
#input_file = '25_week_sample_cleaned_and_mapped.tsv.gz'
#output_file = '25_week_sample_aggregated.tsv.gz'

print('Input file selected: ', input_file)
print('Output file selected', output_file)

Input file selected:  3_day_sample_cleaned_and_mapped.tsv.gz
Output file selected 3_day_sample_aggregated.tsv.gz


In [6]:
##### LOAD DATA
print('Loading data...')

Loading data...


In [7]:
date_columns = ['hit_time_gmt',
                 'date_time']
df = pd.read_csv('../data/processed_data/'+input_file, compression='gzip', sep='\t', encoding='iso-8859-1', quoting=3, low_memory=False, parse_dates=date_columns)

print('Loading data complete.')

Loading data complete.


In [8]:
##### AGGREGATE NUMERICAL COLUMNS
print('Aggregating numerical columns...')

Aggregating numerical columns...


In [9]:
# select numerical columns
numerical_cols_names = ['visitor_id', 
                        'visit_num', 
                        'visit_page_num', 
                        'hit_time_gmt',
                        'date_time',
                        'purchase_boolean', 
                        'product_view_boolean', 
                        'checkout_boolean', 
                        'cart_addition_boolean', 
                        'cart_removal_boolean', 
                        'cart_view_boolean', 
                        'campaign_view_boolean', 
                        'cart_value', 
                        'page_view_boolean', 
                        'last_purchase_num', 
                        'num_product_items_seen', 
                        'sum_price_product_items_seen', 
                        'hit_counter', 
                        'standard_search_results_clicked', 
                        'standard_search_started', 
                        'suggested_search_results_clicked']

numerical_cols = df.loc[:, df.columns.isin(numerical_cols_names)].copy()

# group numerical columns by visitor_id and visit_num and aggregate
numerical_cols_aggregated = numerical_cols.groupby(by = ['visitor_id', 
                                                         'visit_num'], as_index=False).agg({'visit_page_num' : 'max',
                                                                                                          'hit_time_gmt': ['min', 'max'],
                                                                                                          'date_time' : ['min', 'max'],
                                                                                                          'purchase_boolean' : 'sum',
                                                                                                          'product_view_boolean' : 'sum',
                                                                                                          'checkout_boolean' : 'sum',
                                                                                                          'cart_addition_boolean': 'sum',
                                                                                                          'cart_removal_boolean': 'sum',
                                                                                                          'cart_view_boolean': 'sum',
                                                                                                          'campaign_view_boolean': 'sum',
                                                                                                          'cart_value': 'sum',
                                                                                                          'page_view_boolean': 'sum',
                                                                                                          'last_purchase_num': 'max',
                                                                                                          'num_product_items_seen' : 'sum',
                                                                                                          'sum_price_product_items_seen' : 'sum',
                                                                                                          'hit_counter' : 'sum',
                                                                                                          'standard_search_results_clicked' : 'sum',
                                                                                                          'standard_search_started' : 'sum',
                                                                                                          'suggested_search_results_clicked' : 'sum'})

# rename columns
numerical_cols_aggregated.columns = ['_'.join(x) for x in numerical_cols_aggregated.columns.ravel()]
numerical_cols_aggregated = numerical_cols_aggregated.rename(columns={'visitor_id_' : 'visitor_id',
                                                                      'visit_num_' : 'visit_num', 
                                                                      'visit_page_num_max' : 'visit_page_num',
                                                                      'hit_time_gmt_min' : 'hit_time_gmt',
                                                                      'hit_time_gmt_max' : 'last_hit_time_gmt_visit',
                                                                      'date_time_min' : 'date_time',
                                                                      'date_time_max' : 'last_date_time_visit',
                                                                      'purchase_boolean_sum' : 'purchase',
                                                                      'product_view_boolean_sum' : 'product_views',
                                                                      'checkout_boolean_sum' : 'checkouts',
                                                                      'cart_addition_boolean_sum' : 'cart_additions',
                                                                      'cart_removal_boolean_sum' : 'cart_removals',
                                                                      'cart_view_boolean_sum' : 'cart_views',
                                                                      'campaign_view_boolean_sum' : 'campaign_views',
                                                                      'cart_value_sum' : 'cart_value',
                                                                      'page_view_boolean_sum' : 'page_views',
                                                                      'last_purchase_num_max' : 'last_purchase_num',
                                                                      'num_product_items_seen_sum' : 'num_product_items_seen',
                                                                      'sum_price_product_items_seen_sum' : 'sum_price_product_items_seen',
                                                                      'hit_counter_sum' : 'hit_count',
                                                                      'standard_search_results_clicked_sum' : 'standard_search_results_clicked', 
                                                                      'standard_search_started_sum' : 'standard_search_started', 
                                                                      'suggested_search_results_clicked_sum' : 'suggested_search_results_clicked'})

# sort by hit_time_gmt, last_hit_time_gmt_visit, visitor_id and visit_num
numerical_cols_aggregated = numerical_cols_aggregated.sort_values(['hit_time_gmt', 
                                                                   'last_hit_time_gmt_visit', 
                                                                   'visitor_id', 
                                                                   'visit_num'], ascending=[True, True, True, True])

# reset index to make sure that index values are unique
numerical_cols_aggregated = numerical_cols_aggregated.reset_index(drop=True)

print('Aggregating numerical columns complete.')

Aggregating numerical columns complete.


In [10]:
##### PROCESS CATEGORICAL COLUMNS
print('Processing categorical columns...')

Processing categorical columns...


In [11]:
# select categorical columns
categorical_cols_names = ['visitor_id',
                          'hit_time_gmt',  
                          'country', 
                          'cookies', 
                          'persistent_cookie', 
                          'search_page_num',
                          'connection_type', 
                          'browser', 
                          'operating_system', 
                          'search_engine', 
                          'search_engine_generalized',
                          'marketing_channel', 
                          'referrer_type', 
                          'new_visit', 
                          'hourly_visitor', 
                          'daily_visitor', 
                          'weekly_visitor', 
                          'monthly_visitor', 
                          'quarterly_visitor', 
                          'yearly_visitor', 
                          'product_categories_level_1', 
                          'product_categories_level_2', 
                          'product_categories_level_3', 
                          'device_type_user_agent', 
                          'device_brand_name_user_agent', 
                          'device_operating_system_user_agent', 
                          'device_browser_user_agent',
                          'repeat_orders', 
                          'net_promoter_score', 
                          'hit_of_logged_in_user',
                          'registered_user',
                          'user_gender', 
                          'user_age', 
                          'visit_during_tv_spot']

categorical_cols = df.loc[:, df.columns.isin(categorical_cols_names)].copy()

# sort by hit_time_gmt, visitor_id and visit_num
categorical_cols = categorical_cols.sort_values(['hit_time_gmt', 
                                                 'visitor_id'], ascending=[True, True])

# reset index to make sure that index values are unique
categorical_cols = categorical_cols.reset_index(drop=True)

print('Preparing categorical columns complete.')

Preparing categorical columns complete.


In [12]:
##### MERGE NUMERICAL AND CATEGORICAL COLUMNS
print('Merging numerical and categorical columns...')

Merging numerical and categorical columns...


In [13]:
df = pd.merge_asof(numerical_cols_aggregated, categorical_cols, on='hit_time_gmt', by='visitor_id')

# reset index to make sure that index values are unique
df = df.reset_index(drop=True)

print('Merging numerical and categorical columns complete.')

Merging numerical and categorical columns complete.


In [14]:
##### WRITE DATAFRAME TO FILE
print('Writing dataframe to file...')

Writing dataframe to file...


In [15]:
df.to_csv('../data/processed_data/'+output_file, compression='gzip', sep='\t', encoding='iso-8859-1', index=False)

In [16]:
print('Aggregation complete.')
run_time = datetime.now() - start_time
print('Run time: ', run_time)

run_time_dict_file = '3_day_sample_aggregation_run_time.txt'
#run_time_dict_file = '6_week_sample_cleaning_and_mapping_run_time.txt'
#run_time_dict_file = '12_week_sample_cleaning_and_mapping_run_time.txt'
#run_time_dict_file = '25_week_sample_cleaning_and_mapping_run_time.txt'

run_time_dict = {'aggregation run time' : run_time}

f = open('../results/descriptives/'+run_time_dict_file, 'w')
f.write(str(run_time_dict))
f.close()

Aggregation complete.
Run time:  0:01:07.328444
